# Setup

In [4]:
# Weights and Biases
#!pip install -q wandb
# Tensorflow
#!pip install -q tensorflow

In [7]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tensorflow import square, reduce_mean
from tensorflow.keras.losses import MSE

In [2]:
import config
#Erlend
wandb.login(key=config.erlend_key)
# Hjalmar
#wandb.login(key=config.hjalmar_key)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: erlendrygg (avogadro). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Erlend/.netrc


True

# Load, split and normalize data

### Load data

In [3]:
file = "../data/processed_data/2021_2022.csv"
df_read = pd.read_csv(file)
display(df_read)

,Unnamed: 0,Quote_date,Expire_date,Price,Underlying_last,Strike,TTM,Moneyness,Underlying_return,Underlying_1,...,Underlying_82,Underlying_83,Underlying_84,Underlying_85,Underlying_86,Underlying_87,Underlying_88,Underlying_89,Underlying_90,R
0,704239,2021-05-17,2021-05-19,3062.000,4163.6,1100.0,2,3.785091,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.0
1,704240,2021-05-17,2021-05-19,2962.505,4163.6,1200.0,2,3.469667,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.0
2,704241,2021-05-17,2021-05-19,2862.600,4163.6,1300.0,2,3.202769,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.0
3,704242,2021-05-17,2021-05-19,2761.600,4163.6,1400.0,2,2.974000,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.0
4,704243,2021-05-17,2021-05-19,2661.600,4163.6,1500.0,2,2.775733,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.0
5,704244,2021-05-17,2021-05-19,2561.595,4163.6,1600.0,2,2.602250,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.0
6,704245,2021-05-17,2021-05-19,2461.600,4163.6,1700.0,2,2.449176,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.0
7,704246,2021-05-17,2021-05-19,2362.000,4163.6,1800.0,2,2.313111,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.0
8,704247,2021-05-17,2021-05-19,2261.600,4163.6,1900.0,2,2.191368,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.0
9,704248,2021-05-17,2021-05-19,2161.595,4163.6,2000.0,2,2.081800,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.0


### Format input data

In [4]:
df = df_read

# Format settings
max_timesteps = 90
moneyness = False # Moneyness = True WIP
bs_vars = ['Moneyness', 'TTM', 'R'] if moneyness else ['Underlying_last', 'Strike', 'TTM', 'R']
underlying_lags = ['Underlying_last'] + [f'Underlying_{i}' for i in range (1, max_timesteps)]

# Create train, validation and test set split points
train_start = datetime(2021,1,1)
val_start = train_start + relativedelta(months=11)
test_start = val_start + relativedelta(months=1)
test_end = test_start + relativedelta(months=1)
train_start = str(train_start.date())
val_start = str(val_start.date())
test_start = str(test_start.date())
test_end = str(test_end.date())

# Split train and validation data
df_train = df[(df['Quote_date'] >= train_start) & ( df['Quote_date'] < val_start)]
df_val = df[(df['Quote_date'] >= val_start) & ( df['Quote_date'] < test_start)]

# Extract target values
train_y = df_train['Price'].to_numpy()
val_y = df_val['Price'].to_numpy()

# Convert dataframes to numpy arrays
train_x = [df_train[underlying_lags].to_numpy(), df_train[bs_vars].to_numpy()]
val_x = [df_val[underlying_lags].to_numpy(), df_val[bs_vars].to_numpy()]

# Scale features based on training set
underlying_scaler = MinMaxScaler()
train_x[0] = underlying_scaler.fit_transform(train_x[0])
val_x[0] = underlying_scaler.transform(val_x[0])

bs_scaler = MinMaxScaler()
train_x[1] = bs_scaler.fit_transform(train_x[1])
val_x[1] = bs_scaler.transform(val_x[1])

# Shuffle training set
np.random.seed(0)
shuffle = np.random.permutation(len(train_x[0]))
train_x = [train_x[0][shuffle], train_x[1][shuffle]]
train_y = train_y[shuffle]

# Reshape data to fit LSTM
train_x = [train_x[0].reshape(len(train_x[0]), max_timesteps,1), train_x[1]]
val_x = [val_x[0].reshape(len(val_x[0]), max_timesteps, 1), val_x[1]]

print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')

Train shape: (1119088, 90, 1), (1119088, 4)
Val shape: (202710, 90, 1), (202710, 4)


# Model construction

In [ ]:
# WIP
def moneyness_mse(y_true, y_pred):
    '''Moneyness MSE loss function'''
    return MSE(y_true[:,0], y_pred)

# WIP
def scaled_moneyness_mse(y_true, y_pred):
    return MSE(y_true[:,0]*y_true[:,1], y_pred*y_true[:,1])

In [5]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''

    # Input layers
    underlying_history = Input((config.LSTM_timesteps,1))
    bs_vars = Input((config.Num_features,))

    # LSTM layers
    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 1),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        return_sequences = False
    ))

    # MLP layers
    layers = Concatenate()([model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU()(layers)

    output = Dense(1, activation='relu')(layers)

    # Exponential decaying learning rate
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x[0])/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    # Compile model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    model.summary()
    return model

# Hyperparameter search setup

In [ ]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'random',
    'name': 'LSTM-MLP v.1.0 testing : 21-22 data',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [8, 16, 32, 64, 96, 128]},
        'MLP_units': {
            'values': [32, 64, 96, 128]},
        'LSTM_timesteps': {
            'values': [20]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 6, 'min': 2},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 6, 'min': 2},
        'Bn_momentum': {
            'distribution': 'uniform',
            'max': 1, 'min': 0},
        'Lr': {
            'distribution': 'uniform',
            'max': 0.01, 'min': 0.0001},
        'Lr_decay': {
            'distribution': 'uniform',
            'max': 1, 'min': 0.9},        
        'Minibatch_size': {
            'values': [1024, 2048, 4096]},
        'Min_delta': {
            'value': 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 4},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing - test area') 
#sweep_id = '98bxt6oq'

Create sweep with ID: ybzdubg7
Sweep URL: https://wandb.ai/avogadro/Deep%20learning%20for%20option%20pricing%20-%20test%20area/sweeps/ybzdubg7


# Run hyperparameter search

In [6]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Build model and create callbacks
        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )

        # Adapt sequence length to config
        print(config.LSTM_timesteps)
        print(train_x[0].shape, train_x[1].shape)
        train_x[0] = train_x[0][:, :config.LSTM_timesteps, :]
        val_x[0] = val_x[0][:, :config.LSTM_timesteps, :]
        print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
        print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')

        # Train model
        model.fit(
            train_x,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x, val_y),
            epochs = 1000,
            callbacks = [early_stopping, wandb_callback] 
        )

### Run full sweep

In [16]:
wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing - test area', count = 1)

wandb: Agent Starting Run: x1nwz702 with config:
wandb: 	Bn_momentum: 0.2115311974369981
wandb: 	LSTM_layers: 4
wandb: 	LSTM_timesteps: 20
wandb: 	LSTM_units: 64
wandb: 	Lr: 0.007252310655150062
wandb: 	Lr_decay: 0.931385828545152
wandb: 	MLP_layers: 5
wandb: 	MLP_units: 128
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 1024
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


### Single run

In [8]:
config = {
    'LSTM_units': 8,
    'MLP_units': 100,
    'LSTM_timesteps': 90,
    'LSTM_layers': 3,
    'MLP_layers': 4,
    'Bn_momentum': 0.99,
    'Lr': 0.01,
    'Lr_decay': 1,
    'Minibatch_size': 4096,
    'Min_delta': 1,
    'Patience': 20,
    'Num_features': 3 if moneyness else 4, 
    'Architecture': 'LSTM-MLP v.0.1',
    'Dataset': f'Moneyness. Train_start: {train_start}, val_start: {val_start}, {test_start}: {test_start}',
}
trainer(config = config, project = 'Deep learning for option pricing - test area')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 90, 1)]      0           []                               
                                                                                                  
 sequential_1 (Sequential)      (None, 8)            1408        ['input_3[0][0]']                
                                                                                                  
 input_4 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 12)           0           ['sequential_1[0][0]',           
                                                                  'input_4[0][0]']          